In [ ]:
from google.colab import drive
drive.mount("/content/gdrive")
!pwd
%cd "/content/gdrive/Shareddrives/PPEREZ IA"
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/Shareddrives/PPEREZ IA
/content/gdrive/Shareddrives/PPEREZ IA
'Actividad 1.1. Evaluación de las fuentes de búsqueda.gslides'
'Actividad 1.2 Usando base de datos de artículos científicos..gslides'
'Actividad 1.3. Revisando un artículo científico..gslides'
'Actividad 1.4 Seleccionando las bases para el marco de trabajo.gdoc'
'Actividad 1.5. Desarrolla un estado del arte_Equipo 1.gdoc'
'Actividad 4.2: Método Lineal Congruencial y Pruebas de Aleatoriedad.gdoc'
'Actividad 4.2: Método Lineal Congruencial y Pruebas de Aleatoriedad.ipynb'
'Actividad 4.6: Conceptos básicos de Cadenas de Markov.gdoc'
'  Actividad 4.8: Fila de espera con un servidor_Equipo1.gdoc'
'Actividad 4.8: Fila de espera con un servidor_Equipo1.ipynb'
 Archivos
'Artículos de investigación'
'Bloque Anterior TC3006C'
'Bloque Anterior TC3007C'
 CodeClassificationPreproces

In [ ]:
!pip install javalang

In [ ]:
import javalang
from collections import defaultdict
import numpy as np
import pandas as pd
import csv
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import itertools as IT
import re
import math
from scipy.spatial import distance

In [ ]:
def readFile(file):
    """
    Lee el archivo original y lo guarda en una variable.

    Parameters:
    file (str): Ruta del archivo a leer.

    Returns:
    str: Contenido del archivo leído.
    """
    with open(file, 'r', encoding='utf-8') as f:
        fileRead = f.read()
        return fileRead

In [ ]:
def extractComments(java_code):
    """
    Extrae los comentarios de un código Java.

    Parameters:
    java_code (str): Código Java.

    Returns:
    list: Lista de palabras en los comentarios.
    """
    # Expresiones regulares para los comentarios de una línea y de bloque
    singleLineComment = re.compile(r'//.*')
    javadocComment = re.compile(r'/\*\*.*?\*/', re.DOTALL)
    multiLineComment = re.compile(r'/\*(?!\*).*?\*/', re.DOTALL)

    # Encontrar todos los comentarios y sus posiciones
    comments = []
    for match in singleLineComment.finditer(java_code):
        comments.append((match.start(), match.group()))
    for match in javadocComment.finditer(java_code):
        comments.append((match.start(), match.group()))
    for match in multiLineComment.finditer(java_code):
        comments.append((match.start(), match.group()))

    # Ordenar los comentarios por su posición en el código
    comments.sort(key=lambda x: x[0])

    # Extraer y procesar los comentarios en orden
    words = []
    for _, comment in comments:
        cleanedComment = re.sub(r'//|/\*\*|/\*|\*/|\*', '', comment)
        cleanedComment = cleanedComment.lower()
        words_in_comment = re.findall(r'\b[\w@áéíóúÁÉÍÓÚñÑüÜ]+\b', cleanedComment, re.UNICODE)
        words.extend(words_in_comment)

    return words

In [ ]:
def tokenizeJavaCode(javaCode):
    """
    Obtiene una lista de tokens de una cadena de texto del código Java

    Parameters:
    javaCode (str): Código fuente en Java.

    Returns:
    list: Lista de tipos de tokens obtenidos del código.
    """
    tokenArray = []
    tokens = list(javalang.tokenizer.tokenize(javaCode))
    for token in tokens:
        tokenArray.append(token.__class__.__name__)
    return tokenArray


In [ ]:
def parserJavaCodeAst(javaCode):
    """
    Parsea el código fuente de Java en un árbol AST.

    Parameters:
    javaCode (str): Código fuente de Java.

    Returns:
    javalang.ast.Node: Raíz del árbol AST generado.
    """
    return javalang.parse.parse(javaCode)


In [ ]:
def createTransitions(tree):
    """
    Crea una matriz de transiciones a partir del árbol AST de Java.

    Parameters:
    tree (javalang.ast.Node): Raíz del árbol AST.

    Returns:
    dict: Diccionario de transiciones de nodos.
    """
    transitions = defaultdict(lambda: defaultdict(int))
    traverse(tree, transitions)
    return transitions

In [ ]:
def traverse(node, transitions):
    """
    Recorre el árbol AST de Java y registra las transiciones entre los nodos.

    Parameters:
    node (javalang.ast.Node): Nodo actual del AST.
    transitions (dict): Diccionario que guarda las transiciones de nodos.
    """
    if isinstance(node, javalang.ast.Node):
        nodeType = type(node).__name__
        children = node.children
        for child in children:
            if isinstance(child, (list, javalang.ast.Node)):
                if isinstance(child, list):
                    for sub_child in child:
                        traverse(sub_child, transitions)
                else:
                    childType = type(child).__name__
                    transitions[nodeType][childType] += 1
                    traverse(child, transitions)
            elif child is not None:  # Handle non-Node children
                childType = type(child).__name__
                transitions[nodeType][childType] += 1

In [ ]:
def buildProbablityTransitionMatrix(transitions):
    """
    Crea una matriz de transiciones basada en la probabilidad de ocurrencia de
    cada transición de un nodo.

    Parameters:
    transitions (dict): Diccionario de transiciones de tokens.

    Returns:
    np.array: Matriz de probabilidad de transiciones.
    """
    transitionMatrix = {}

    for nodeSource, destinations in transitions.items():
        total = sum(destinations.values())
        if total == 0:
            continue

        probabilityDestinationCounts = {}
        for destination, count in destinations.items():
            probabilityDestinationCounts[destination] = count / total

        transitionMatrix[nodeSource] = probabilityDestinationCounts

    #Creación de una matriz basada en un arreglo de arreglos para
    #la creación de una matriz m*m, recorriendo una lista de la matriz
    #de transiciones originales
    nodeTypes = list(transitionMatrix.keys())
    matrix = []
    for node1 in nodeTypes:
        subMatrix = []
        for node2 in nodeTypes:
            subMatrix.append(transitionMatrix[node1].get(node2, 0))
        matrix.append(subMatrix)

    return np.array(matrix)

In [ ]:
def padMatrixMatch(matrix1, matrix2):
    """
    Ajusta dos matrices a la misma dimensión rellenando con ceros
    las filas y columnas faltantes.

    Parameters:
    matrix1 (np.array): Primera matriz.
    matrix2 (np.array): Segunda matriz.

    Returns:
    tuple: Dos matrices ajustadas a la misma dimensión.
    """
    rows1, cols1 = matrix1.shape
    rows2, cols2 = matrix2.shape

    if rows1 > rows2:
        matrix2 = np.pad(matrix2, ((0, rows1 - rows2), (0, 0)), mode='constant')
    elif rows2 > rows1:
        matrix1 = np.pad(matrix1, ((0, rows2 - rows1), (0, 0)), mode='constant')

    if cols1 > cols2:
        matrix2 = np.pad(matrix2, ((0, 0), (0, cols1 - cols2)), mode='constant')
    elif cols2 > cols1:
        matrix1 = np.pad(matrix1, ((0, 0), (0, cols2 - cols1)), mode='constant')

    return matrix1, matrix2

In [ ]:
def cosineSimilarity(matrix1, matrix2):
    """
    Calcula la similitud por coseno de dos matrices dadas.

    Parameters:
    matrix1 (np.array): Primera matriz.
    matrix2 (np.array): Segunda matriz.

    Returns:
    float: Similitud por coseno entre las dos matrices.
    """
    matrix1, matrix2 = padMatrixMatch(matrix1, matrix2)

    # Calcular el producto punto de la matriz y su transpuesta
    dotProduct = np.trace(np.dot(matrix1, matrix2.T))

    # Calcular las magnitudes de las matrices
    magnitudeA = np.trace(np.dot(matrix1.T, matrix1))
    magnitudeB = np.trace(np.dot(matrix2.T, matrix2))

    # Calcular el coseno del ángulo
    cosTheta = dotProduct / (np.sqrt(magnitudeA) * np.sqrt(magnitudeB))

    return cosTheta


In [ ]:
def cosineSimilarityMeandistribution(text1, text2):
    """
    Calcula la similitud coseno basada en la media de distribución de palabras
    entre dos textos.

    Parameters:
    text1 (list): Lista de palabras del primer texto.
    text2 (list): Lista de palabras del segundo texto.

    Returns:
    float: Similitud por coseno entre las distribuciones de palabras de los dos textos.
    """
    #Reescribir la cadena en minúsculas
    combineText = text1 + text2
    text1Dicc = {}
    text2Dicc = {}
    idf = []
    tf1 = []
    tf2 = []

    # Inicializar los diccionarios de frecuencia
    for i in range(len(combineText)):
        text1Dicc[combineText[i]] = 0
        text2Dicc[combineText[i]] = 0

    # Contar las frecuencias de palabras en cada texto
    for j in range(len(text1)):
        text1Dicc[text1[j]] = text1Dicc.get(text1[j], 0) + 1

    for k in range(len(text2)):
        text2Dicc[text2[k]] = text2Dicc.get(text2[k], 0) + 1

    # Calcular TF
    tf1Value = sum(text1Dicc.values())
    tf2Value = sum(text2Dicc.values())

    for key in text1Dicc:
        tf1.append(text1Dicc.get(key)/tf1Value)

    for key in text2Dicc:
        tf2.append(text2Dicc.get(key)/tf2Value)

    # Calcular IDF
    for key in text1Dicc:
        if text1Dicc.get(key) > 0 and text2Dicc.get(key)  > 0:
            idf.append(math.log(2/(2+1))+1)
        else:
            idf.append(math.log(2/(1+1))+1)

    # Calcular TF-IDF
    for i in range(len(tf1)):
        tf1[i] = tf1[i] * idf[i]
        tf2[i] = tf2[i] * idf[i]

    # Calcular el producto punto
    dotProduct = np.dot(tf1, tf2)

    # Calcular las magnitudes de los vectores
    magnitudeA = np.linalg.norm(tf1)
    magnitudeB = np.linalg.norm(tf2)

    # Calcular el coseno del ángulo
    cosTheta = dotProduct/ (magnitudeA * magnitudeB)

    return cosTheta

In [ ]:
def cosineSimilaritiesComments(comments1, comments2):
    """
    Calcula las similitudes coseno para los comentarios entre dos conjuntos de comentarios.

    Parameters:
    comments1 (list): Lista de palabras en los comentarios del primer conjunto.
    comments2 (list): Lista de palabras en los comentarios del segundo conjunto.

    Returns:
    tuple: Un par de valores de similitud coseno: media de distribución y similitud de Markov.
    """
    cosineMeanDist = 0
    cosineMarkov = 0
    if not comments1 or not comments2:
        return cosineMeanDist, cosineMarkov
    else:
        cosineMeanDist = cosineSimilarityMeandistribution(comments1, comments2)
        if len(comments1) > 1 or len(comments2) > 1:
          return cosineMeanDist, cosineMarkov
        else:
          transitionComments1 = createTransitionsTokens(comments1)
          transitionComments2 = createTransitionsTokens(comments2)
          transitionCommentsMatrix1 = buildProbablityTransitionMatrix(transitionComments1)
          transitionCommentsMatrix2 = buildProbablityTransitionMatrix(transitionComments2)
          cosineMarkov = cosineSimilarity(transitionCommentsMatrix1,transitionCommentsMatrix2)

        return cosineMeanDist, cosineMarkov

In [ ]:
def createTransitionsTokens(tokens):
    """
    Construye la matriz de transiciones a partir de la lista de tokens.

    Parameters:
    tokens (list): Lista de tipos de tokens.

    Returns:
    dict: Diccionario de transiciones de tokens.
    """
    transitions = defaultdict(lambda: defaultdict(int))
    #Obtener las transiciones del nodo actual al siguiente nodo
    #Guardarlo en un diccionario de diccionarios contando el número de ocurrencias
    for i in range(len(tokens) - 1):
        currentToken = tokens[i]
        nextToken = tokens[i + 1]
        transitions[currentToken][nextToken] += 1
    return transitions

In [ ]:
def euclideanDistance(matrix1, matrix2):
    """
    Calcula la distancia euclidiana de dos matrices dadas.

    Parameters:
    matrix1 (np.array): Primera matriz.
    matrix2 (np.array): Segunda matriz.

    Returns:
    float: Valor de la distancia euclidiana entre las dos matrices.
    """
    return distance.euclidean(matrix1, matrix2)

In [ ]:
def manhattanDistance(matrix1, matrix2):
    """
    Calcula la distancia de Manhattan de dos matrices dadas.

    Parameters:
    matrix1 (np.array): Primera matriz.
    matrix2 (np.array): Segunda matriz.

    Returns:
    float: Valor de la distancia de Manhattan entre las dos matrices.
    """
    return distance.cityblock(matrix1, matrix2)

In [ ]:
def jaccardDistance(matrix1, matrix2):
    """
    Calcula la distancia de jaccard de dos matrices dadas.

    Parameters:
    matrix1 (np.array): Primera matriz.
    matrix2 (np.array): Segunda matriz.

    Returns:
    float: Valor de la distancia de jaccard entre las dos matrices.
    """
    return distance.jaccard(matrix1, matrix2)

In [ ]:
def vectorDifference(vector1, vector2):
    """
    calcula la diferencia de los 2 vectores previamente obtenidos.

    Parameters:
    vector1 (np.array): Primer vector.
    vector2 (np.array): Segundo vector.

    Returns:
    float: Valor del vector1 menos vector2
    """
    differenceVector = vector1 - vector2
    return differenceVector

In [ ]:
def preProcessData(pairs,veredicts):
    """
    Abre los archivos correspondientes y calcula las caracteristicas
    importantes de ambos codigos.

    Parameters:
    pairs (pd.dataframe): Nombres combinados del par de codigos (c1_c2).
    veredicts (pd.dataframe): Valor binario del veredicto
                              (1 es plagio, 0 no es plagio).

    Returns:
    x: Arreglo con las caracteristicas importantes de ambos codigos.
    y: Veredicto de plagio entre ambos codigos.
    """
    x = []
    y = []
    for pair, veredict in zip(pairs, veredicts):
        code1, code2 = pair.split("_")
        if code1 is not None and code2 is not None and veredict is not None:
            # Obtener el contenido de los archivos
            print(code1)
            print(code2)
            code1_file = readFile(f"./JAVA CODES/{code1}.java")
            code2_file = readFile(f"./JAVA CODES/{code2}.java")
            #-------------------------------------------------------------
            #Comentarios
            comments = extractComments(code1_file)
            comments2 = extractComments(code2_file)
            cosSimMeanDistComments, cosSimMarkovComments = cosineSimilaritiesComments(comments, comments2)

            # ------------------------------------------------------------
            # Obtener los tokens de los archivos
            code1_tokens = tokenizeJavaCode(code1_file)
            code2_tokens = tokenizeJavaCode(code2_file)
            # Obtener diccionario de transición de tokens
            code1_transitionTokens = createTransitionsTokens(code1_tokens)
            code2_transitionTokens = createTransitionsTokens(code2_tokens)

            code1_tranTokensMatriz = buildProbablityTransitionMatrix(code1_transitionTokens)
            code2_tranTokensMatriz = buildProbablityTransitionMatrix(code2_transitionTokens)

            cosSimTokens = cosineSimilarity(code1_tranTokensMatriz,code2_tranTokensMatriz)
            # ------------------------------------------------------------
            # Obtener el ast de los archivos
            code1_ast = parserJavaCodeAst(code1_file)
            code2_ast = parserJavaCodeAst(code2_file)
            # Obtener el diccionaro de transiciones
            code1_transitions = createTransitions(code1_ast)
            code2_transitions = createTransitions(code2_ast)
            # Obtener la matriz de transiciones basada en probabilidad
            code1_tranMatriz = buildProbablityTransitionMatrix(code1_transitions)
            code2_tranMatriz = buildProbablityTransitionMatrix(code2_transitions)
            # Hacer que las matrices anteriores tengan el mismo tamañano
            code1_adjustedTransitionMatriz, code2_adjustedTransitionMatriz = padMatrixMatch(code1_tranTokensMatriz,code2_tranTokensMatriz)
            # Obtener la similaridad de coseno entre ambas matrices
            cosSim = cosineSimilarity(code1_adjustedTransitionMatriz,code2_adjustedTransitionMatriz)
            # "Aplanar" las matrices ajustadas
            vector1 = code1_adjustedTransitionMatriz.flatten()
            vector2 = code2_adjustedTransitionMatriz.flatten()
            # Obtener la distancia euclidiana
            euclidian = euclideanDistance(vector1,vector2)
            # Obtener la distancia de Manhattan
            manhattan = manhattanDistance(vector1,vector2)
            # Obtener la distancia de jaccard
            jaccard = jaccardDistance(vector1,vector2)
            # Obtener la diferencia de vectores (matriz1-matriz2)
            differenceVector = vectorDifference(vector1,vector2)
            # Guardar las caracteristicas importantes de los codigos recibidos
            tempX = []
            tempX.append(cosSimMeanDistComments)
            tempX.append(cosSimMarkovComments)
            tempX.append(cosSimTokens)
            tempX.append(cosSim)
            tempX.append(euclidian)
            tempX.append(manhattan)
            tempX.append(jaccard)
            tempX.append(differenceVector)
            x.append(tempX)
            tempY = []
            tempY.append(int(veredict))
            y.append(tempY)
    return x,y

In [ ]:
def readTrainTest():
    """
    Obtiene la información correspondiente de los archivos de train y test,
    llama a la función encargada del preprocesado de la información y
    almacena los resultados en un archivo csv.
    """
    # Obtener la información correspondiente a train
    df_train = pd.read_csv("./Dataset/train.csv", delimiter=',')
    pairs_train = df_train['pairs'].tolist()
    veredicts_train = df_train['verdict'].tolist()
    train_x, train_y = preProcessData(pairs_train,veredicts_train)
    # Obtener la información correspondiente a test
    df_test = pd.read_csv("./Dataset/test.csv", delimiter=',')
    pairs_test = df_test['pairs'].tolist()
    veredicts_test = df_test['verdict'].tolist()
    test_x, test_y = preProcessData(pairs_test,veredicts_test)
    # Almacenar los resultados de train
    column_names = ['cosSimMeanDistComments','cosSimMarkovComments','cosSimTokens','cosSim', 'euclidean', 'manhattan', 'jaccard', 'differenceVector']

    df_X_train = pd.DataFrame(train_x, columns=column_names)
    df_X_test = pd.DataFrame(test_x, columns=column_names)
    df_y_train = pd.DataFrame(train_y, columns=['veredict'])
    df_y_test = pd.DataFrame(test_y, columns=['veredict'])

    df_X_train.to_csv("./Dataset/Procesado2/x_train.csv", index=False)
    df_X_test.to_csv("./Dataset/Procesado2/x_test.csv", index=False)
    df_y_train.to_csv("./Dataset/Procesado2/y_train.csv", index=False)
    df_y_test.to_csv("./Dataset/Procesado2/y_test.csv", index=False)

In [ ]:
readTrainTest()

464a03b8
ff1fc018
3e6def38
548ffb07
213340b3
35f0c004
7bc92b7f
9028caf7
6f393cfe
b185d034
1dab88fb
bac616ee
115c99cb
921b6e4a
5be7547e
cb032314
4e9c4bf9
7011024d
680ba922
6e207cbf
14b0fb8e
8ddb5587
16a2a867
3951966f
bf0df1d5
ea899386
b2001d68
ba468e1f
35eb27da
64ce7b1e
5aebbb1b
e647bef7
30e0cc81
b55888de
05f939b3
e647bef7
a4d6775d
cb87df79
317a209c
6b97058e
3088ca9c
7a9c69d8
6946f466
ce7027ee
80881cae
9555b9d1
24afd00e
810fd282
6bcc5afd
734a94be
04ed33a5
6b83b22e
c1fef98f
f870e76b
949502c2
9fc811f7
4ea10951
66e74577
cdb801a1
ed610dc9
752ea9a5
b2ec0eff
6f02c6d9
ff34fab2
1be078c4
bb0949b3
01c915a2
480de7be
6be98ece
e1c4f3db
2e1109d7
e7000ac2
1097b326
a60fba84
3d06b643
6bcc5afd
6f02c6d9
884f5678
0d4d22e0
f898cee0
3dd65549
f229aa7f
44428e63
9debf95c
18e2441c
43b10dec
3a318b43
90b71536
3e6def38
e7dce35b
6bcc5afd
f6ca6fc8
7c11511f
ca8f11a4
1984bef1
2063ba3e
0df4050e
f5fde094
6e7cd58b
cfba313d
2f8c3bf3
a3abac01
66e74577
6e207cbf
0017d438
9852706b
0c173033
6b83b22e
24044589
e435b1ac
8535bdf7
e

Hasta este momento:

*   Similitud de coseno
*   Distancia euclidiana
*   Distancia de Manhattan
*   Distancia de jaccard
*   Diferencia de vectores
